In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
from numpy import sqrt

In [2]:
df1 = pd.read_csv('Listings.csv')
df2 = pd.read_csv('Listings2.csv')
df3 = pd.read_csv('Listings3.csv')
df4 = pd.read_csv('Listings4.csv')
full_df = pd.concat([df1,df2,df3,df4], ignore_index=True)
#full_df.head()

# <font color=blue size=6> 1. DATA CLEANING & PREPROCESSING

## Checking for Nan or empty entries

In [3]:
#plt.figure(figsize=[15,10])
#sns.heatmap(full_df.isna())

In [4]:
#full_df[full_df.Price_psf.isna()].head()

In [5]:
correct_df = full_df[full_df.Area_Sqft.str.contains('sqft')]
wrong_df = full_df[full_df.Area_Sqft.str.contains('psf')]
#------------------------------
wrong_df['Price_psf'] = wrong_df['Area_Sqft']
wrong_df['Area_Sqft'] = wrong_df['NumBaths']
wrong_df['NumBaths'] = np.NAN
#-------------------------------
full_df = pd.concat([wrong_df, correct_df], ignore_index=True)
#full_df.head()

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

## Let's proceed to transform the numerical columns from string forms into numeric form

In [6]:
def strip_num(text):
    number_lst = re.findall('\\b\\d*[.,]*\\d+\\b', text)
    assert len(number_lst) == 1
    number = float(number_lst[0].replace(',',''))
    return number

In [7]:
# Checking for weird numbers (those that have more than one ',' or '.' in itself)
for field in ['Area_Sqft','Price_psf','RentalRate']:
    for i in range(len(full_df)):
        try:
            haha=strip_num(full_df[field][i])
        except:
            #print('Silly Entry!!!!!!!!! Index = ' , i)
            pass
#full_df.iloc[4379]

### This lousy entry has
1. Exaggerated area. Perhaps the agent quote the area of the entire compound
2. Price_psf equals 0. This is not possible
3. Very abnormally cheap price/month for a condo unit

### such an entry should not be trusted. I will drop it

In [8]:
full_df.drop(index=4379, inplace=True)
#-------------------------------
full_df.Area_Sqft = full_df.Area_Sqft.apply(strip_num)
full_df.Price_psf = full_df.Price_psf.apply(strip_num)
full_df.RentalRate = full_df.RentalRate.apply(strip_num)

## Checking for duplicated entries
* We recognize these duplicates by referring to their Urls

In [9]:
dups=full_df[full_df.duplicated(subset='Url', keep=False)].sort_values(by='Url')
#print('There are {} duplicated entries'.format(len(dups)))
#dups.head(6)

In [10]:
full_df.drop_duplicates(subset='Url', inplace=True)

## Let's recheck the Nan or empty values

In [11]:
#plt.figure(figsize=[15,10])
#sns.heatmap(full_df.isna())

## Now the numeric columns are fully filled. Let's move on or take a look at other columns, especially those with empty entries

In [12]:
#print(list(full_df.District.unique()),'\n')
#print(list(full_df.Ethnic.unique()),'\n')
#print(list(full_df.Facing.unique()),'\n')
#print(list(full_df.Floor.unique()),'\n')
#print(list(full_df.Furnishing.unique()),'\n')
#print(list(full_df.Keys_On_Hand.unique()),'\n')
#print(list(full_df.Lease.unique()),'\n')
#print(list(full_df.NumBaths.unique()),'\n')
#print(list(full_df.NumBeds.unique()),'\n')
#print(list(full_df.Pets.unique()),'\n')

## <font color=blue> We encounter the following interesting things</font>
<font size=3>
1. The columns' nan entries can first be replaced with **"No Mention"**
2. There are a few of weird entries in which the rental place has **'None Beds'**???
3. Studio should be **grouped with** '1 bed'?
4. What is up with the places that has no mentioning of **NumBaths**. Can we find a way to fill in the info?
5. **Furnishing & District** looks like columns where we can try to fill in the empty entries. Other columns have way to many to handle
4. The 'Floor' column definitely requires **more cleaning & standardizing** in case we **need to use** it later

## Let's perform action number one first

In [13]:
full_df.fillna(value='No Mention', inplace=True)

## Let's perform further cleaning for the NumBeds feature

In [14]:
none_beds=full_df[full_df.NumBeds == 'None Beds']
none_beds.Description.apply(lambda x: 'studio' in x.lower())

2549     True
2572    False
4483     True
4494     True
4508     True
4731     True
4740     True
4741     True
4769     True
4850     True
Name: Description, dtype: bool

In [15]:
#none_beds.Description[2572][:20]

In [16]:
full_df.NumBeds.replace(to_replace='None Beds', value='1 Bed', inplace=True)
full_df.NumBeds[2572] = '3 Beds'
full_df.NumBeds.replace(to_replace='Studio', value='1 Bed', inplace=True)
full_df.NumBeds = full_df.NumBeds.apply(lambda x: int(re.findall('\d+',x)[0]))

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Let's look at entries where NumBaths is not mentioned

In [17]:
def find_bathroom_info(text):
    ans=re.findall('\\d+ bath[a-z]*', text.lower())
    fin_ans = '0'
    if len(ans)!=0:
        if ans[0][0]=='1':
            fin_ans = ans[0].title()
        else:
            fin_ans = (ans[0]+'s').title()
    return fin_ans

In [18]:
full_df.NumBaths = list(map(lambda x,y: find_bathroom_info(y) if x=='No Mention' else x, full_df.NumBaths, full_df.Description))
full_df.NumBaths = full_df.NumBaths.apply(lambda x: int(re.findall('\d+', x)[0]))

In [19]:
#plt.figure(figsize=[15,10])
#sns.heatmap(full_df.applymap(lambda x: x==0))

In [20]:
grouped_numbed=full_df[full_df.NumBaths!=0][['NumBeds','NumBaths']].groupby(by='NumBeds').mean()
dict1 = {grouped_numbed.index[i]:int(round(grouped_numbed.NumBaths.iloc[i])) for i in range(len(grouped_numbed))}

In [21]:
full_df.NumBaths = list(map(lambda x,y: dict1[y] if x==0 else x, full_df.NumBaths, full_df.NumBeds))
#full_df.NumBaths.unique()

## Let's look at entries where Furnishing is not mentioned

In [22]:
def find_furnishing_info(text):
    ans=re.findall('[?:A-Za-z]+[?: .]*furnish[a-z]*', text.lower())
    fin_ans = 'No Mention'
    if len(ans)!=0:
        fin_ans = ans[0].title()
    return fin_ans

#ser2 = full_df[full_df.Furnishing=='No Mention'].Description.apply(find_furnishing_info)
#ser2.where(ser2.apply(len)>=1).dropna().unique()

In [23]:
#ser2.where((ser2=="With Furnished") | (ser2=="And Furnished") |
#           (ser2=="Be Furnished") | (ser2=="Between Furnished")).dropna()

In [24]:
#check=full_df.Description[[837,1026,3497,4024]]
#for i in range(4):
#    print(check.iloc[i])
#    print('########################################')

In [25]:
full_df.Furnishing = list(map(lambda x,y: find_furnishing_info(y) if x=='No Mention' else x, 
                              full_df.Furnishing, full_df.Description))
full_df.Furnishing.replace('Partial Furnished', 'Partially Furnished', inplace=True)
full_df.Furnishing.replace('Partial Furnish', 'Partially Furnished', inplace=True)
full_df.Furnishing.replace('Parital Furnish', 'Partially Furnished', inplace=True)
full_df.Furnishing.replace('Bare Furnish', 'Partially Furnished', inplace=True)
full_df.Furnishing.replace('Full Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Nicely Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Luxuriously Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Tastefully Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Stylishly Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Newly Furnish', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('And Furnished', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Full Furnish', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Fully Furnish', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Full Furnishing', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Luxurious Furnishing', 'Fully Furnished', inplace=True)
full_df.Furnishing.replace('Be Furnished', 'Flexible', inplace=True)
full_df.Furnishing.replace('Between Furnished', 'Flexible', inplace=True)
full_df.Furnishing.replace('With Furnished', 'Flexible', inplace=True)
full_df.Furnishing.replace('Flexible Furnishing', 'Flexible', inplace=True)
full_df.Furnishing.replace('Easy Furnishing', 'Flexible', inplace=True)
full_df.Furnishing.replace('Unfurnish', 'Unfurnished', inplace=True)

## Let's look at entries where District is not mentioned

In [26]:
def find_district_info(text):
    ans=re.findall('district(?: )*\d+', text.lower())
    fin_ans = 'No mention'
    if len(ans)!=0:
        fin_ans = ans[0].title().replace('District ','')
        fin_ans = 'D'+(2-len(fin_ans))*'0'+fin_ans
    return fin_ans

In [27]:
full_df.District = list(map(lambda x,y: find_district_info(y) if x=='No Mention' else x, full_df.District, full_df.Description))

In [28]:
#plt.figure(figsize=[15,10])
#sns.heatmap(full_df.applymap(lambda x: "No Mention" in str(x)))

## Let's look at entries where Amenities are not mentioned

In [29]:
amenities = []
for i in range(len(full_df)):
    amenities += full_df.Amenities.iloc[i].split(',')
amenities_list=list(set(amenities))
amenities_list.sort()
#print(amenities_list)

In [30]:
def find_amenities_info(text):
    ans=re.findall('[a-zA-z, :]*(?:amenities|facilities)[a-zA-z, :]*', text.lower())
    fin_ans = ''
    if len(ans)!=0:
        fin_ans = ans[0].title()
    return fin_ans

#ser4 = full_df[full_df.Amenities == 'No Mention'].Description.apply(find_amenities_info)
#ser4.where(ser4.apply(len)>=1).dropna()

In [31]:
#print(full_df.Description[4639])
#print('###############################')
#print(full_df.Description[2418])

### Only data extracted from description at index 340,341,4639,2418 are valid for updating of the Amenities col

In [32]:
full_df.Amenities[340:342] = (full_df.Description[340:342].apply(find_amenities_info)
                                     .apply(lambda x: x.replace('Amenities Include ',''))
                                     .apply(lambda x: x.replace(' And',','))
                                     .apply(lambda x: x.replace(' Bbq Pit','BBQ'))
                                     .apply(lambda x: x.replace(', ',',')))
full_df.Amenities[4639] = 'Aircon,Swimming Pool,Playground,Gymnasium,Function Room,BBQ,Steam Room,High Ceiling,Maid Room,Garden,Outdoor Dining'
full_df.Amenities[2418] = 'Aircon' 

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [33]:
#full_df.Amenities[[340,341,4639,2418]]

In [34]:
with_amenities = full_df[full_df.Amenities != 'No Mention']
without_amenities = full_df[full_df.Amenities == 'No Mention']

In [35]:
def find_amenities_info_2(amenities_list,text):
    regex='|'.join(amenities_list)
    ans=re.findall(regex, text.title())
    fin_ans = ''
    if len(ans)!=0:
        fin_ans = ans[0].title()
    return fin_ans

In [36]:
without_amenities.Amenities=without_amenities.Description.apply(lambda x: find_amenities_info_2(amenities_list, x))
full_df = pd.concat([with_amenities, without_amenities], sort=True)

C:\Users\admin\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [37]:
## Finally let's drop some highly sparse columns

In [38]:
full_df.drop(labels=['Ethnic','Facing','Floor','Keys_On_Hand','Pets'], axis=1, inplace=True)

## Final Dataframe after filling in some missing values

In [39]:
#plt.figure(figsize=[15,10])
#sns.heatmap(full_df.applymap(lambda x: "No Mention" in str(x)))

## Extracting info about the property type

In [40]:
full_df['PropertyType'] = list(map(lambda x: re.findall('hdb|landed|condo', x)[0], full_df.Url))
#full_df['PropertyType'].unique()

In [41]:
full_df.to_csv('Cleaned_combined_Listings.csv')